In [15]:
from tkinter import *
from tkinter import ttk
import smtplib
import random
import math
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import re

def limit_char6(str):
    return len(str) <= 6

def login_home():
    global frame

    frame = ttk.Frame(root)
    frame.pack(fill = BOTH, pady=20)

    frame_top = ttk.Frame(frame)
    frame_top.pack(expand = True)

    frame_bottom = ttk.Frame(frame)
    frame_bottom.pack(expand = True)

    label = ttk.Label(frame_top, text='ユーザー名')
    label.pack(side = LEFT)

    entry = ttk.Entry(frame_top)
    entry.pack(side = RIGHT)

    label2 = ttk.Label(frame_bottom, text='パスワード')
    label2.pack(side = LEFT)

    entry2 = ttk.Entry(frame_bottom, show='*')
    entry2.pack(side = RIGHT)
    ###パスワードは半角英数、記号だけで構成されているかどうかを確認する必要あり。

    button = ttk.Button(frame, text='ログイン', command=login_panel)
    button.pack()

    label3 = ttk.Label(frame, text="", foreground='red')
    label3.pack()
    ###上のラベルはログインに失敗した場合に「ユーザー名またはパスワードが違います」を表示するよう

    link = ttk.Label(frame, text='パスワードを忘れた場合はこちら',foreground='blue',cursor='hand1',font=('MSゴシック', 12, 'underline'))
    link.pack()
    link.bind("<Button-1>",lambda e:forget_pass())

    link2 = ttk.Label(frame, text='新規登録はこちら',foreground='blue',cursor='hand1',font=('MSゴシック',12,'underline'))
    link2.pack()
    link2.bind("<Button-1>",lambda e:make_account())

def frame_destroy(frame_name):
    frame_name.destroy()
    login_home()

def forget_pass():
    """
    ここはパスワードを忘れた場合である。

    ユーザー名に対応したメールアドレス（または入力してもらったシステムに使用しているメールアドレス)に仮パスワード（認証コード)を送る
    仮パスワードをシステムに入力してもらい、本人確認が取れたらパスワード変更画面に遷移
    パスワード変更完了
    """
    global frame_forget

    frame.destroy()

    frame_forget = ttk.Frame(root)
    frame_forget.pack(fill = BOTH, pady = 20)

    label_forget1 = ttk.Label(frame_forget, text='ユーザー名またはメールアドレス')
    label_forget1.pack()

    entry_forget1 = ttk.Entry(frame_forget)
    entry_forget1.pack()

    button_forget1 = ttk.Button(frame_forget, text='パスワードを変更する', command=lambda:mail_check(entry_forget1.get()))
    button_forget1.pack()

    ttk.Button(frame_forget, text="ログイン画面に戻る", command=lambda:frame_destroy(frame_forget)).pack(side=BOTTOM)
    ###登録されているユーザー名またはメールアドレスと一致するかどうかを確認する
    ###メールで確認コードを送信して、メールの確認を行う。コードはランダムな6桁の数字とかがいい？

def mail_check(txt):
    """
    とりあえず確認用に入力されたメールアドレスにメールを送信するプログラムを作成する。
    本当であれば、ユーザー名、メールアドレスが見つからない場合の処理も行う必要がある。
    """
    smtp_server = "smtp.gmail.com"
    port = 587

    server = smtplib.SMTP(smtp_server, port)
    server.starttls()

    login_address = "p.seminar.vts@gmail.com"
    login_password = "aoxorgtkblyaptjc"

    server.login(login_address, login_password)

    message = MIMEMultipart()

    message["Subject"] = "【Virtual Trading System】パスワード変更のお知らせ"
    message["From"] = login_address
    message["To"] = txt

    rand = math.floor(1 + (random.random() * 9)) * 100000 + math.floor(random.random() * 10) * 10000 + math.floor(random.random() * 10) * 1000 + math.floor(random.random() * 10) * 100 + math.floor(random.random() * 10) * 10 + math.floor(random.random() * 10)
    text = MIMEText("パスワードの再設定をご希望の場合は次の認証コードをアプリケーションに入力し、パスワードの変更を行ってください。\n\n認証コード:"+str(rand) + "\n\n本メールにお心当たりのない場合は、お手数ですが本メールの破棄をお願いいたします。")
    message.attach(text)

    server.send_message(message)

    server.quit()


    frame_forget.destroy()
    ###確認コード入力画面を作成
    global CheckCodeFrame_forget

    CheckCodeFrame_forget = ttk.Frame(root)
    CheckCodeFrame_forget.pack(fill = BOTH, pady = 20)

    CheckCodeLabel_forget = ttk.Label(CheckCodeFrame_forget, text='入力したメールアドレスに届いた確認コードを入力してください')
    CheckCodeLabel_forget.pack()

    vc = root.register(limit_char6)
    CheckCodeEntry_forget = ttk.Entry(CheckCodeFrame_forget, validate='key',validatecommand=(vc, "%P"), width=7)
    CheckCodeEntry_forget.pack()

    CheckCodeButton_forget = ttk.Button(CheckCodeFrame_forget, text="確認する", command=lambda:checkCode_forget(str(rand), CheckCodeEntry_forget.get()))
    CheckCodeButton_forget.pack()

def checkCode_forget(rand, received):
    if rand == received:
        ###ここにコードが正しい場合の処理(パスワード変更画面に遷移)を書く
        print("correct")
    else:
        ###ここにコードが間違っていた場合の処理を書く(確認コードが間違っています、程度を出力すれば問題ないと思われる。)
        ###入力ミスが一定回数を超えた場合にやり直させる。
        print("missed")

def make_account():
    """
    ここは、新規アカウントの作成を行う場合である。

    メールアドレスの入力（すでに使われているメールアドレスでないかどうかの確認)
    入力してもらったメールアドレスに仮パスワードを送信
    仮パスワードを入力してもらい、本人確認が取れたら本パスワードを作ってもらう。（半角英数字、記号、文字数制限)（2回同じパスワードを入力してもらう)
    登録完了
    """
    global frame_make_account
    global label_make_account1

    frame.destroy()

    frame_make_account = ttk.Frame(root)
    frame_make_account.pack(fill = BOTH, pady = 20)

    label_make_account = ttk.Label(frame_make_account, text="メールアドレスを入力してください。")
    label_make_account.pack()

    entry_make_account = ttk.Entry(frame_make_account, width=65)
    entry_make_account.pack()

    button_make_account = ttk.Button(frame_make_account, text="メールを送信する",command=lambda:makeAccountCheckCode(entry_make_account.get()))
    button_make_account.pack()

    label_make_account1 = ttk.Label(frame_make_account, text="", foreground='red')
    label_make_account1.pack()

    ttk.Button(frame_make_account, text="ログイン画面に戻る", command=lambda:frame_destroy(frame_make_account)).pack(side=BOTTOM)

def makeAccountCheckCode(address):
    """
    ここで、メールアドレスが正しいかどうかを最初に確認し、正しくない場合は（@が含まれていないなど)不正であることを伝える。
    そして、正しい場合は確認コードを載せたメールをそのメールアドレスに送信し、確認コードが正しいことを確認する。
    確認コードが正しいことが確認できたら、次のパネルに行く。
    """
    check = True
    re_isTrue = re.compile(r'^[a-zA-Z0-9_+-]+(.[a-zA-Z0-9_+-]+)*@([a-zA-Z0-9][a-zA-Z0-9-]*[a-zA-Z0-9]*\.)+[a-zA-Z]{2,}$')
    if re.match(re_isTrue, address) == None:
        label_make_account1["text"]="不正なメールアドレスです。"
        check = False
    
    ###使用されているメールアドレスかどうかを確認する。(label_make_account1["text"] = "そのメールアドレスはすでに使用されています。")
    if check:
        frame_make_account.destroy()
        ###メールの送信、確認コードの確認など
        smtp_server = "smtp.gmail.com"
        port = 587

        server = smtplib.SMTP(smtp_server, port)
        server.starttls()

        login_address = "p.seminar.vts@gmail.com"
        login_password = "aoxorgtkblyaptjc"

        server.login(login_address, login_password)

        message = MIMEMultipart()

        message["Subject"] = "【Virtual Trading System】新規登録のお知らせ"
        message["From"] = login_address
        message["To"] = address

        rand = math.floor(1 + (random.random() * 9)) * 100000 + math.floor(random.random() * 10) * 10000 + math.floor(random.random() * 10) * 1000 + math.floor(random.random() * 10) * 100 + math.floor(random.random() * 10) * 10 + math.floor(random.random() * 10)
        text = MIMEText("会員登録のお手続きありがとうございます。\nこの度はVirtual Trading Systemに仮登録していただき、ありがとうございます。\n\n現在の段階では会員登録手続きは完了しておりません。会員登録を完了するには次の認証コードをアプリケーションに入力し、ユーザー名とパスワードを設定してください。\n\n認証コード:"+str(rand) + "\n\n本メールにお心当たりのない場合は、お手数ですが本メールの破棄をお願いいたします。")
        message.attach(text)

        server.send_message(message)

        server.quit()

        global CheckCodeFrame_make

        CheckCodeFrame_make = ttk.Frame(root)
        CheckCodeFrame_make.pack(fill = BOTH, pady = 20)

        CheckCodeLabel_make = ttk.Label(CheckCodeFrame_make, text='入力したメールアドレスに届いた確認コードを入力してください')
        CheckCodeLabel_make.pack()

        vc = root.register(limit_char6)
        CheckCodeEntry_make = ttk.Entry(CheckCodeFrame_make, validate='key',validatecommand=(vc, "%P"), width=7)
        CheckCodeEntry_make.pack()

        CheckCodeButton_make = ttk.Button(CheckCodeFrame_make, text="確認する", command=lambda:checkCode_make(str(rand), CheckCodeEntry_make.get(), address))
        CheckCodeButton_make.pack()

def checkCode_make(rand, text, address):
    if rand==text:
        ###確認コードが正しい場合の処理
        ###ユーザー名とパスワードの設定をする。パスワードは2回記述させる。
        CheckCodeFrame_make.destroy()

        global setting_frame
        setting_frame = ttk.Frame(root)
        setting_frame.pack(fill = BOTH, pady = 20)

        setting_label1 = ttk.Label(setting_frame, text="ユーザー名")
        setting_label1.grid(row=0, column=0)

        setting_entry1 = ttk.Entry(setting_frame)
        setting_entry1.grid(row=0, column=1, sticky=EW, padx=20)

        setting_label2 = ttk.Label(setting_frame, text="パスワード")
        setting_label2.grid(row=1, column=0)

        setting_entry2 = ttk.Entry(setting_frame)
        setting_entry2.grid(row=1, column=1, sticky=EW, padx=20)

        setting_label3 = ttk.Label(setting_frame, text="パスワード(確認用)")
        setting_label3.grid(row=2, column=0)

        setting_entry3 = ttk.Entry(setting_frame)
        setting_entry3.grid(row=2, column=1, sticky=EW, padx=20)

        setting_button = ttk.Button(setting_frame, text="登録する")
        setting_button.grid(row=3, column=0, columnspan=2)

        setting_label4 = ttk.Label(setting_frame, text="",foreground="red")
        setting_label4.grid(row=4, column=0, columnspan=2)

        setting_frame.grid_columnconfigure(1, weight=1)

    else:
        ###確認コードが間違っている場合の処理
        print("Missed")

def registar_check():
    ###ここでは登録情報のチェックを行い、正しく登録できる場合は登録完了のむねをメールと画面で伝えるようにする。
    ###登録情報のチェックとしては(1)ユーザー名が使用されていないか、(2)パスワードの文字数や使用文字が適切か(3)パスワードの入力欄とパスワード(確認)の入力欄の文字列が一致するか、がある。
    print("registar check")

def login_panel():
    """
    ここではログインボタンを押された時に行う処理をかく。

    ユーザー名とパスワードの組のタプルが存在するかどうかをSQliteで確認して、もし正しい組が存在するのであればログインする。
    存在しないのであれば、間違っていることをlabel3["text"]="ユーザー名またはパスワードが違います"で知らせる。
    """

root = Tk()
root.geometry('450x200')
root.title('Virtual Trading System')

login_home()
root.mainloop()